# iALM Stage 2: Liability Projection

This notebook projects household cash outflows. The model distinguishes between:
1. **Fixed Liabilities:** Nominal value is fixed (e.g. Mortgages).
2. **Indexed Liabilities:** Grow with stochastic inflation (e.g. Living costs, School fees).

In [ ]:
import sys
import os
import numpy as np
import matplotlib.pyplot as plt

sys.path.append(os.path.abspath(os.path.join('..', 'src')))
from ialm.engines import iALM_Simulator
from ialm.liabilities import LiabilitySimulator

# Generate Inflation Paths first (Drivers)
sim = iALM_Simulator(dt=1/12)
inflation_paths = sim.simulate_gou(R0=1.03, alpha=0.03, beta=0.5, sigma=0.015, n_steps=240, n_scenarios=100)

## 1. Defining the Liability Stack
We define a "Mortgage" (Fixed) and "School Fees" (Inflation + 2% Spread).

In [ ]:
liab_sim = LiabilitySimulator(inflation_paths)

# A. Fixed Mortgage: £2,000 for 10 years
mortgage = liab_sim.add_liability(
    name="Mortgage", 
    initial_value=2000, 
    start_step=0, 
    end_step=120, 
    is_indexed=False
)

# B. Indexed School Fees: Starts in year 5, lasts 5 years. CPI + 2% spread.
school_fees = liab_sim.add_liability(
    name="School Fees", 
    initial_value=1500, 
    start_step=60, 
    end_step=120, 
    is_indexed=True, 
    r_add=0.02/12
)

## 2. Visualizing Liability Uncertainty
While the mortgage is a straight line, the school fees vary significantly by scenario due to compounding inflation uncertainty.

In [ ]:
# Calculate Means
mean_mortgage = np.mean(mortgage, axis=1)
mean_school = np.mean(school_fees, axis=1)

# Plot
plt.figure(figsize=(12, 6))

# Stack plot for EXPECTED values
plt.stackplot(range(240), mean_mortgage, mean_school, labels=['Mortgage (Fixed)', 'School Fees (Indexed)'], alpha=0.7)

# Overlay uncertainty for School Fees
# We shift the uncertainty cloud up by the mortgage amount so it sits on top of the stack
school_uncertainty = school_fees + mean_mortgage[:, np.newaxis]
plt.plot(school_uncertainty[:, :20], color='grey', alpha=0.1)  # Plot 20 scenarios

plt.title("Projected Liability Cashflows: Fixed vs Indexed Risk")
plt.xlabel("Months")
plt.ylabel("Monthly Outflow (£)")
plt.legend(loc='upper right')
plt.grid(True, alpha=0.3)
plt.show()